In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [20]:
# Generate URLs to parse for data
# All data parsed from armor info on Fextralife
base_url = 'https://eldenring.wiki.fextralife.com/'
data_pieces = ['Helms', 'Chest+Armor', 'Gauntlets', 'Leg+Armor'] # sub-pages we want to pull data from
urls = [base_url + p for p in data_pieces]
urls

['https://eldenring.wiki.fextralife.com/Helms',
 'https://eldenring.wiki.fextralife.com/Chest+Armor',
 'https://eldenring.wiki.fextralife.com/Gauntlets',
 'https://eldenring.wiki.fextralife.com/Leg+Armor']

In [21]:
# Pull data and use BeautifulSoup to parse
parsed_pages =[] # For storing parsed armor dataframes
for url_idx, url in enumerate(urls):
    print(url)
    class_data = requests.get(url) # Data for specific class
    parser = BeautifulSoup(class_data.content, parser='lxml') # Generate parser
    table = parser.findAll('table')[0] # Find table with armor info
    rows = table.findAll('tr') # Get rows of table
    parsed_rows = [] # For storing parsed rows
    for idx, row in enumerate(rows): # Iterate
        if idx == 0: # Handle header row
            row_items = row.findAll('th')
        else: # Data rows
            row_items = row.findAll('td')     
        # Add parsed data
        parsed_rows.append([c.get_text(strip=True).replace(u'\xa0', '') for c in row_items])
    # Create DataFrame from results
    colnames = [c.replace(' ', '').replace('.', '') for c in parsed_rows[0]] # Clean up columns
    print(colnames)
    parsed_frame = pd.DataFrame(data=parsed_rows[1:], columns=colnames)
    parsed_frame['ArmorType'] = data_pieces[url_idx] # Add in flag for armor type
    parsed_pages.append(parsed_frame)

https://eldenring.wiki.fextralife.com/Helms
['Name', 'Phy', 'VSStr', 'VSSla', 'VSPie', 'Mag', 'Fir', 'Lit', 'Hol', 'Imm', 'Rob', 'Foc', 'Vit', 'Poi', 'Wgt', 'SpecialEffect', 'HowtoAcquire', 'In-GameSection']
https://eldenring.wiki.fextralife.com/Chest+Armor
['Name', 'Phy', 'VSStr', 'VSSla', 'VSPie', 'Mag', 'Fir', 'Lit', 'Hol', 'Imm', 'Rob', 'Foc', 'Vit', 'Poi', 'Wgt', 'SpecialEffect', 'HowtoAcquire', 'In-GameSection']
https://eldenring.wiki.fextralife.com/Gauntlets
['Name', 'Phy', 'VSStr', 'VSSla', 'VSPie', 'Mag', 'Fir', 'Lit', 'Hol', 'Imm', 'Rob', 'Foc', 'Vit', 'Poi', 'Wgt', 'SpecialEffect', 'HowtoAcquire', 'In-GameSection']
https://eldenring.wiki.fextralife.com/Leg+Armor
['Name', 'Phy', 'VSStr', 'VSSla', 'VSPie', 'Mag', 'Fir', 'Lit', 'Hol', 'Imm', 'Robu', 'Foc', 'Vita', 'Poi', 'Wgt', 'SpecialEffect', 'HowtoAcquire', 'In-GameSection']


In [22]:
# Concat data
# Note - due to issues in leg armor naming conventions we will modify it's column names before concatenating
parsed_pages[-1].columns = parsed_pages[0].columns # Update Leg Armor column names
armor_frame = pd.concat(parsed_pages)
armor_frame['ArmorType'] = armor_frame['ArmorType'].str.strip().replace('\W', '', regex=True)
armor_frame.head()

,Name,Phy,VSStr,VSSla,VSPie,Mag,Fir,Lit,Hol,Imm,Rob,Foc,Vit,Poi,Wgt,SpecialEffect,HowtoAcquire,In-GameSection,ArmorType
0,Alberich's Pointed Hat,1.8,1.4,1.8,1.8,4.6,4.2,4.4,4.6,16,10,29,31,2,1.7,Strengthens thorn sorcery,The entireAlberich's Setcan be found in the ma...,2,Helms
1,Alberich's Pointed Hat (Altered),0.9,0.2,0.9,0.9,4.4,3.8,4.0,4.4,12,7,23,24,1,1.0,,Boc the Seamstercan alter the originalAlberich...,2,Helms
2,Albinauric Mask,4.0,3.1,4.0,3.8,2.5,3.1,2.1,2.5,12,23,10,10,5,3.8,"+4Arcane,reduces theHPrecovery effects of theF...",Found inVolcano Manornear theGuest Hallgrace. ...,4,Helms
3,All-Knowing Helm,4.6,4.2,4.8,4.2,4.4,3.4,3.6,3.1,12,20,9,9,7,4.6,,The entireAll-Knowing Setis dropped bySir Gide...,9,Helms
4,Aristocrat Hat,3.1,3.1,2.8,3.1,3.8,4.0,3.8,3.1,22,14,18,20,4,3.0,,Drops fromSoldier Aristocratsthat can be found...,1,Helms


In [23]:
# Expor data
armor_frame.to_csv('EldenRing_Armor_Data.txt', sep='|', index=False)